# Extracting data from job desc

In [2]:
with open("job_desc.md") as f:
    job_desc = f.read()


In [48]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_message = SystemMessagePromptTemplate.from_template(
    "You're an expert career consultant with an IQ over 140 working with a special client regarding this job posting."
    "Please list the skills and requirements for this job.")
job_desc_prompt = HumanMessagePromptTemplate.from_template("Here is the job description:\n{description}")
chat_prompt = ChatPromptTemplate.from_messages([system_message, job_desc_prompt])

messages = chat_prompt.format_prompt(description=job_desc).to_messages()

In [5]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=.75, model_name="gpt-3.5-turbo")

In [6]:
response = chat_model.predict_messages(messages)

In [7]:
response

AIMessage(content='The core skills defined in the job description are:\n\n1. Python and Django: 5+ years of experience building complex web applications using Python and Django framework.\n2. Software Engineering Principles: Hands-on experience designing software and breaking down complexity using software engineering principles.\n3. React: Experience with React framework or similar.\n4. Code Quality: Ability to write well-structured and documented code.\n5. Product Development: Continuous updating and expansion of products.\n6. Multinational Teamwork: Ability to be an important part of a multinational, agile software development team.\n7. Communication: Working knowledge of English for effective communication.\n8. Fast-paced Environment: Ability to thrive in a fast-paced environment and quick learner.\n9. Results-driven: Results-driven mindset and ability to prioritize activities with the greatest impact.\n10. Ownership and Excellence: Takes ownership and demands excellence from thems

## Formatting w/ pydantic

In [8]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

class Skills(BaseModel):
    skills: List[str]

In [40]:
from langchain.prompts import PromptTemplate

In [55]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Skills)
format_instructions = parser.get_format_instructions()

format_prompt = PromptTemplate(template="{format_instructions}",
                               partial_variables={'format_instructions': format_instructions},
                               input_variables=[])

format_prompt = SystemMessagePromptTemplate(input_variables=[], prompt=format_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_message, format_prompt, job_desc_prompt])


In [56]:
chat_prompt

ChatPromptTemplate(input_variables=['description'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You're an expert career consultant with an IQ over 140 working with a special client regarding this job posting.Please list the skills and requirements for this job.", template_format='f-string', validate_template=True), additional_kwargs={}), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]

In [57]:
messages = chat_prompt.format_prompt(description=job_desc).to_messages()

output = chat_model.predict_messages(messages)

parser.parse(output.content)
output

AIMessage(content='{\n  "skills": [\n    "Python",\n    "Django",\n    "React",\n    "Snowflake",\n    "Software engineering principles",\n    "Agile software development",\n    "Web application development",\n    "Code documentation",\n    "Teamwork",\n    "Fast learner",\n    "Results-driven",\n    "English language proficiency"\n  ]\n}', additional_kwargs={}, example=False)

In [ ]:
chat_prompt

In [66]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI

prompt = PromptTemplate(template="""
    You're an expert career consultant with an IQ over 140 working with a special client regarding this job posting.
    Please list the skills and requirements for this job.\n{format_instructions}
    Here is the job description:\n{description}
""",
                        input_variables=["description"],
                        partial_variables={'format_instructions': format_instructions})
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo')
chain = LLMChain(prompt=chat_prompt, llm=llm, output_parser=parser)

/Users/swe/repos/resume_slayer/venv/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/swe/repos/resume_slayer/venv/lib/python3.11/site-packages/langchain/llms/openai.py:801: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [67]:
chain.predict(description=job_desc)

Skills(skills=['Python', 'Django', 'React', 'Snowflake', 'Software Engineering', 'Teamwork', 'Fast Learner', 'Results-driven', 'English'])